## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import google_api_key

# Configure gmaps API key
gmaps.configure(api_key=google_api_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Hilo,US,19.7297,-155.0900,77.00,65,90,8.05,light rain
1,1,Atuona,PF,-9.8000,-139.0333,78.37,77,52,14.70,broken clouds
2,2,Gao,ML,16.6362,1.6370,72.88,17,95,11.36,overcast clouds
3,3,Gallup,US,35.5281,-108.7426,55.40,16,1,13.80,clear sky
4,4,Victoria,HK,22.2855,114.1577,55.00,51,96,10.47,overcast clouds


In [14]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Hilo,US,19.7297,-155.0900,77.00,65,90,8.05,light rain
1,1,Atuona,PF,-9.8000,-139.0333,78.37,77,52,14.70,broken clouds
2,2,Gao,ML,16.6362,1.6370,72.88,17,95,11.36,overcast clouds
7,7,Japura,ID,-0.3167,102.3500,72.37,97,100,2.66,overcast clouds
8,8,Cape Town,ZA,-33.9258,18.4232,71.60,60,0,17.27,clear sky


In [18]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID        159
City           159
Country        159
Lat            159
Lng            159
Max Temp       159
Humidity       159
Cloudiness     159
Wind Speed     159
Description    159
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Hilo,US,19.7297,-155.0900,77.00,65,90,8.05,light rain
1,1,Atuona,PF,-9.8000,-139.0333,78.37,77,52,14.70,broken clouds
2,2,Gao,ML,16.6362,1.6370,72.88,17,95,11.36,overcast clouds
7,7,Japura,ID,-0.3167,102.3500,72.37,97,100,2.66,overcast clouds
8,8,Cape Town,ZA,-33.9258,18.4232,71.60,60,0,17.27,clear sky
...,...,...,...,...,...,...,...,...,...,...
683,683,Tangkak,MY,2.2673,102.5453,77.00,78,20,8.05,few clouds
684,684,Kabo,CF,7.6994,18.6290,77.22,19,6,5.03,clear sky
691,691,Yenagoa,NG,4.9247,6.2642,77.95,85,27,5.64,scattered clouds
695,695,Bucerias,MX,20.7500,-105.3333,71.60,78,20,10.36,few clouds


In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", 
                     "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Hilo,US,77.00,light rain,19.7297,-155.0900,
1,Atuona,PF,78.37,broken clouds,-9.8000,-139.0333,
2,Gao,ML,72.88,overcast clouds,16.6362,1.6370,
7,Japura,ID,72.37,overcast clouds,-0.3167,102.3500,
8,Cape Town,ZA,71.60,clear sky,-33.9258,18.4232,
9,Palmas,BR,78.80,moderate rain,-10.2128,-48.3603,
14,Nkoteng,CM,74.95,scattered clouds,4.5167,12.0333,
17,East London,ZA,70.79,overcast clouds,-33.0153,27.9116,
23,Mount Isa,AU,75.20,scattered clouds,-20.7333,139.5000,
24,Gourcy,BF,75.43,scattered clouds,13.2167,-2.3500,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_api_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head()

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Hilo,US,77.00,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
1,Atuona,PF,78.37,broken clouds,-9.8000,-139.0333,Villa Enata
2,Gao,ML,72.88,overcast clouds,16.6362,1.6370,
7,Japura,ID,72.37,overcast clouds,-0.3167,102.3500,Rumah mbah Jamah
8,Cape Town,ZA,71.60,clear sky,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"


In [24]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

In [25]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F </dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info )
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))